## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
from pymystem3 import Mystem
m = Mystem()

import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df['dob_years'].unique()
df.isnull().sum()
df.count()
df.info()
#print(df.loc[df['dob_years'] == 0].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В нашей таблице присутствуют отрицательные значения трудового стажа; образование местами введено курсивом; трудовой стаж в днях записан не целыми числами; где-то значения трудового стажа вызывают подозрения количеством дней; цели кредитов одинаковы, но записаны по разному; местами возраст равен 0; есть пропуски

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#df.info()
#df.isnull().sum()
# Пропуски в данных и совпадение количества пропусков в днях и доходе могут означать, что человек никогда не работал и не
# получал доход, либо ошибся в вводе и пропустил данные, либо безработный. 
# Количество пропусков - 2174.Это чуть больше 10% от общего количества.
# Такое количество данных удалять нельзя, поэтому заменю из медианами по каждой категории профессии.
df['days_employed'] = abs(df['days_employed'])
df['children']=abs(df['children'])
df['income_type'].unique()
for row in df['income_type'].unique():
    median_income = df.loc[df['income_type'] == row, 'total_income'].median()
    df.loc[(df['income_type'] == row)&(df['total_income'].isna()), 'total_income'] = median_income
for row in df['income_type'].unique():
    median_income = df.loc[df['income_type'] == row, 'days_employed'].median()
    df.loc[(df['income_type'] == row)&(df['days_employed'].isna()), 'days_employed'] = median_income
df.info()
# df['dob_years'].unique()
# print(df.loc[df['dob_years'] == 0].count())
# пропуски также есть и в годах, но их всего 101, что составляет слишком малый %, их можно не изменять

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Теперь у нас в таблице нет пропусков. По этим данным можно анализировать.
Медианные значения были выбраны мной по каждой категории профессии. Такой подход я выбрал, чтобы не терять людей, которые ошиблись, но и доход их был сопоставим их должности или статусу (безработный, пенсионер)

### Замена типа данных

In [3]:
df['education']=df['education'].str.lower()
df['family_status']=df['family_status'].str.lower()
df['purpose']=df['purpose'].str.lower()
df.loc[df['gender'] == 'XNA', 'gender'] = 'M'
def employed_years(days):
    if 0 <= days <= 2190:
        return 'без опыта'
    if 2190 < days <= 21900:
        return 'опытный'
    if days > 21900:
        return 'подозрительные данные'
df['years'] = df['days_employed'].apply(employed_years)
print(df.loc[df['years'] == 'подозрительные данные'].count())

children            3858
days_employed       3858
dob_years           3858
education           3858
education_id        3858
family_status       3858
family_status_id    3858
gender              3858
income_type         3858
debt                3858
total_income        3858
purpose             3858
years               3858
dtype: int64


In [19]:
def employed_years(days):
    if 0 <= days <= 2190:
        return 'без опыта'
    if 2190 < days <= 21900:
        return 'опытный'
    if days > 21900:
        return 'подозрительные данные'
df['years'] = df['days_employed'].apply(employed_years)
print(df.loc[df['years'] == 'подозрительные данные'])
df.loc[df['days_employed'] > 300000, 'days_employed'] /= 24
df['days_employed']=df['days_employed'].astype('int64')
df['total_income']=df['total_income'].astype('int64')
print(df.loc[df['years'] == 'подозрительные данные'].count())

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, years, purpose_lem, purpose_one]
Index: []
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years               0
purpose_lem         0
purpose_one         0
dtype: int64


In [5]:
# Я узнал, что происходит с данными по дням в стаже.За основу взял расчет, что есть люди с опытом работы менее полугода (180 дней) 
# или "без опыта", от полугода до 60 лет трудового стажа или "опытный" и более 60 лет - "подозрительные данные". 
# Методом count() я узнал, что подозрительных данных по этой группе 3858.
# Скорее всего данные были указаны просто некорректно. Была выбрана неправильная мера измерения.
# Секунды и минуты дали бы мне слишком маленькие значения, значит буду пробовать часы (разделить на 24)
# Данный расчет позволил мне получить подходящие значения у пенсионеров с "подозрительными данными"

In [6]:
df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

In [7]:
print(df.loc[df['children'] == 20])
print(df.loc[df['children'] == 20].count())

       children  days_employed  dob_years education  education_id  \
606          20            880         21   среднее             1   
720          20            855         44   среднее             1   
1074         20           3310         56   среднее             1   
2510         20           2714         59    высшее             0   
2941         20           2161          0   среднее             1   
...         ...            ...        ...       ...           ...   
21008        20           1240         40   среднее             1   
21325        20            601         37   среднее             1   
21390        20           1547         53   среднее             1   
21404        20            494         52   среднее             1   
21491        20            173         27   среднее             1   

         family_status  family_status_id gender income_type  debt  \
606    женат / замужем                 0      M   компаньон     0   
720    женат / замужем           

In [8]:
# 20 детей встречается у 76 человек, однако среди них много людей с небольшим возрастом
# Скорее всего была допущена ошибка при заполнении
# так как 76 человек - ничтожно малая часть от общей таблицы, то заменим все данные по 20 детям на 2

In [9]:
df['children']=df['children'].replace(20, 2)
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [10]:
# Подозрительные данные отсутствуют

### Вывод

Теперь все подозрительные данные отсутствуют. Наша таблица готова к анализу

### Обработка дубликатов

In [11]:
print('Количество дублей до удаления:', df.duplicated().sum())

Количество дублей до удаления: 71


In [12]:
df.drop_duplicates(inplace=True)
print('Количество дублей после удаления:', df.duplicated().sum())

Количество дублей после удаления: 0


### Вывод

Удалив дубликаты, получили таблицу для анализа. Такие дубликаты могли возникнуть, если человек подавал два заявления на кредит.

### Лемматизация

In [13]:
def purpose_lem(purpose):
    lemmas = ''.join(m.lemmatize(purpose))
    return lemmas 
df['purpose_lem'] = df['purpose'].apply(purpose_lem)
#print(df['purpose_lem'].unique())
def purpose_one(purpose):
    if 'жилье' in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'образование' in purpose:
        return 'образование'
    if 'недвижимость' and 'коммерческий' in purpose:
        return 'бизнес'
    if 'коммерческий' or 'недвижимость' in purpose:
        return 'недвижимость'
df['purpose_one'] = df['purpose_lem'].apply(purpose_one)
print(df['purpose_one'].unique())
from collections import Counter
display(Counter(df['purpose_lem']))

['недвижимость' 'автомобиль' 'образование' 'свадьба' 'бизнес']


Counter({'покупка жилье\n': 646,
         'приобретение автомобиль\n': 461,
         'дополнительный образование\n': 460,
         'сыграть свадьба\n': 765,
         'операция с жилье\n': 652,
         'образование\n': 447,
         'на проведение свадьба\n': 768,
         'покупка жилье для семья\n': 638,
         'покупка недвижимость\n': 621,
         'покупка коммерческий недвижимость\n': 661,
         'покупка жилой недвижимость\n': 606,
         'строительство собственный недвижимость\n': 635,
         'недвижимость\n': 633,
         'строительство недвижимость\n': 619,
         'на покупка подержать автомобиль\n': 478,
         'на покупка свой автомобиль\n': 505,
         'операция с коммерческий недвижимость\n': 650,
         'строительство жилой недвижимость\n': 624,
         'жилье\n': 646,
         'операция со свой недвижимость\n': 627,
         'автомобиль\n': 972,
         'заниматься образование\n': 408,
         'сделка с подержанный автомобиль\n': 486,
         'получ

### Вывод

Применив лемматизацию, я выяснил основные потребности для получения кредита: авто, жилье, свадьба, образование, покупка коммерческой недвижимости (бизнес)

### Категоризация данных

In [14]:
income=pd.cut(df['total_income'], [0, 90000, 135000, 195000, 2300000])
dob=pd.cut(df['dob_years'], [18, 35, 100])

Вывод

Максимальный доход - 2265604
Минимальный - 20667
Задаю основные диапазоны, по которым буду проводить исследования: возраст (от 18 до 35, потому что молодые, от 35 и выше); доход (до 90000 - маленький, до 135000 - средний, до 195000 - выше среднего, свыше 195000 - богатые). Это поможет узнать: для чего берут кредит, есть ли просрочки, как дети и семейное положение влияют на кредит.

### Шаг 3. Ответьте на вопросы

Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
children_group_count = df.groupby('children')['debt'].count()
children_group_sum = df.groupby('children')['debt'].sum()
# общее количество просрочек по каждой категории детей
children_sum_count = children_group_sum / children_group_count * 100
print(children_sum_count)
# группируем по количеству детей в просрочках по кредиту,
# находим сумму всех просрочек и смотрим их отношение к количеству просрочек

df_pivot_children = df.pivot_table(index = ['children'], columns = ['debt'], values = ['purpose_one'], aggfunc='count')
df_pivot_children = df_pivot_children.reset_index()
df_pivot_children.columns = ['children', 'count', 'sum']
df_pivot_children['ratio'] = df_pivot_children['sum'] / df_pivot_children['count'] * 100
display(df_pivot_children)

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


,children,count,sum,ratio
0,0,13028.0,1063.0,8.159349
1,1,4410.0,445.0,10.090703
2,2,1926.0,202.0,10.488058
3,3,303.0,27.0,8.910891
4,4,37.0,4.0,10.810811
5,5,9.0,NaN,NaN


### Вывод

Количество детей влияет на количество кредитов. Чем меньше детей - тем больше берут кредиты.
Вероятность просрочки выше у людей с детьми, чем у бездетных.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
family_status_sum_count = df.groupby('family_status')['debt'].sum() / df.groupby('family_status')['debt'].count() * 100
print(family_status_sum_count)

df_pivot_family = df.pivot_table(index=['family_status'], columns=['debt'], values=['total_income'], aggfunc='count')
df_pivot_family = df_pivot_family.reset_index()
df_pivot_family.columns = ['family_status', 'count', 'sum']
df_pivot_family['ratio'] = df_pivot_family['sum'] / df_pivot_family['count'] * 100
display(df_pivot_family)

family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
не женат / не замужем    9.750890
Name: debt, dtype: float64


,family_status,count,sum,ratio
0,в разводе,1110,85,7.657658
1,вдовец / вдова,896,63,7.031250
2,гражданский брак,3763,388,10.310922
3,женат / замужем,11408,931,8.160940
4,не женат / не замужем,2536,274,10.804416


### Вывод

Хуже всего кредиты отдают люди, находящиеся в гражданском браке, и не женат/не замужем. Скорее всего это связано с тем, что люди подвержены частым изменениям в жизни (появление пары). 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
income=pd.cut(df['total_income'], [0, 90000, 135000, 195000, 2300000])
dob=pd.cut(df['dob_years'], [18, 35, 100])
df_pivot_income = df.pivot_table(index=[income], columns=['debt'], values=['education_id'], aggfunc='count')
df_pivot_income = df_pivot_income.reset_index()
df_pivot_income.columns = ['income', 'count', 'sum']
df_pivot_income['ratio'] = df_pivot_income['sum'] / df_pivot_income['count'] * 100
display(df_pivot_income)

,income,count,sum,ratio
0,"(0, 90000]",3086,262,8.489955
1,"(90000, 135000]",5108,468,9.162099
2,"(135000, 195000]",6484,622,9.592844
3,"(195000, 2300000]",5035,389,7.725919


### Вывод

Кредиты не возвращают все. Чаще всего - люди со средним и выше среднего дохода. 

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
df_pivot_purpose = df.pivot_table(index=['purpose_one'], columns=['debt'], values=['dob_years'], aggfunc='count')
df_pivot_purpose = df_pivot_purpose.reset_index()
df_pivot_purpose.columns = ['purpose', 'count', 'sum']
df_pivot_purpose['ratio'] = df_pivot_purpose['sum'] / df_pivot_purpose['count'] * 100
display(df_pivot_purpose)

,purpose,count,sum,ratio
0,автомобиль,3903,403,10.325391
1,бизнес,1212,99,8.168317
2,недвижимость,8817,683,7.746399
3,образование,3643,370,10.156464
4,свадьба,2138,186,8.699719


### Вывод

Чаще всего не возвращают кредит за автомобиль и образование. Следом идет свадьба и бизнес. Эти категории в чём-то схожи: не всегда затраты на такие мероприятия продуманы и, следовательно, могут потребовать гораздо больше денег, которых будет не хватать. Меньше всего проблем с недвижимостью. Вероятно это связано с тем, что это очень серьезное вложение и люди никак не хотят допустить его потери.

### Шаг 4. Общий вывод

Изучив закономерности, пришел к выводу, что больше всего на просрочки по кредиту влияет именно наличие детей. Возможно, ребенок выступает гарантом, что для человека такой кредит осознанный, он знает, на что идет.
Семейное положение тоже влияет, но в меньшей степени. Находясь в гражданском браке и без пары, люди чаще всего совершали просрочки.
Очень влияет цель кредита. А именно покупка автомобиля. Вероятно связано это с тем, что автомобиль в кредит стараются взять все, вот только такие кредиты порой очень каторжные.
Доход же не оказывает сильное влияние. Просрочки совершают все